In [1]:
"""
EOD HD fundamentals builder — v7

Outputs the columns:
• MACD (12,26,9)   ← raw MACD
• MACD Signal      ← 9-day EMA of MACD
• RSI (1 yr)       ← Wilder's 14-day RSI with current price
• ATH % Chg, Recent-High % Chg, Fair-Value metrics, etc.

NEW FEATURES (v7 update) 4-30-2025:
• Historical price pull now appends real-time price using EODHD's /real-time endpoint
• MACD and Signal values are calculated manually using EMA logic (not via API)
• RSI is calculated using Wilder's method on up-to-date price series
• Timestamp of real-time price is also returned in output

Ticker is printed without the “.US” suffix.
"""

import datetime as _dt
import requests  as _rq
import pandas    as _pd

API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL  = "https://eodhd.com/api"
HEADERS   = {"User-Agent": "EOD-Fundamentals-Client/7.0"}

def _get_json(url: str):
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

def _pct_from_high(curr, high):
    if curr is None or high in (None, 0):
        return None
    return (curr - high) / high

def _pct_to_value(curr, val):
    if curr in (None, 0) or val is None:
        return None
    return (val - curr) / curr

def _strip_suffix(ticker: str) -> str:
    return ticker.rsplit(".", 1)[0]

def _annual_eps_dict(fund: dict) -> dict[int, float]:
    return {
        int(k[:4]): rec["epsActual"]
        for k, rec in fund.get("Earnings", {}).get("Annual", {}).items()
        if rec.get("epsActual") is not None
    }

def _sum_quarterly_eps(hist: list[dict]) -> dict[int, float]:
    eps_q = {}
    for rec in hist:
        eps = rec.get("epsActual")
        d   = _pd.to_datetime(rec.get("date") or rec.get("reportDate"), errors="coerce")
        if _pd.notna(d) and eps is not None:
            eps_q.setdefault(d.year, []).append(eps)
    return {y: sum(v) for y, v in eps_q.items() if len(v) == 4 and sum(v) != 0}

def _five_year_pe_series(df_price: _pd.DataFrame, eps_year: dict[int, float]) -> list[float]:
    pe = []
    for y in sorted(eps_year)[-5:]:
        yr_px = df_price[df_price["date"].dt.year == y]
        if not yr_px.empty and (eps := eps_year[y]) != 0:
            pe.append(yr_px.iloc[-1]["adjusted_close"] / eps)
    return pe

def CurrentPrice(ticker):
    url = f"{BASE_URL}/real-time/{ticker}?api_token={API_TOKEN}&fmt=json"
    d = _get_json(url)
    return d.get("close") or d.get("price") or d.get("lastPrice")

def ATH(ticker):
    today = _dt.date.today().strftime("%Y-%m-%d")
    url   = f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}&from=1900-01-01&to={today}&adjusted=1&fmt=json"
    df    = _pd.DataFrame(_get_json(url))
    return None if df.empty else df["adjusted_close"].max()

def get_historical_prices(ticker, start="2023-01-01"):
    url = (
        f"{BASE_URL}/eod/{ticker}?from={start}&to={_dt.date.today()}"
        f"&api_token={API_TOKEN}&fmt=json"
    )
    response = _rq.get(url)
    data = response.json()
    df = _pd.DataFrame(data)
    df['date'] = _pd.to_datetime(df['date'])
    df.sort_values('date', inplace=True)
    df.set_index('date', inplace=True)
    return df

def get_current_price(ticker):
    url = f"{BASE_URL}/real-time/{ticker}?api_token={API_TOKEN}&fmt=json"
    response = _rq.get(url)
    data = response.json()
    price = data.get("close")
    timestamp = data.get("timestamp")
    if timestamp:
        timestamp = _dt.datetime.fromtimestamp(timestamp)
    return price, timestamp

def append_today_price(df, today_price):
    today = _pd.Timestamp(_dt.date.today())
    if today not in df.index:
        df.loc[today] = df.iloc[-1]  # Copy last known row
        df.loc[today, 'close'] = today_price
    return df

def calculate_rsi_wilder(df, period=14):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df[['close', 'RSI']]

def calculate_macd(df, fast=12, slow=26, signal=9):
    df['EMA_fast'] = df['close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    df['Histogram'] = df['MACD'] - df['Signal']
    return df[['close', 'MACD', 'Signal', 'Histogram']]

def Fundamentals(ticker: str) -> _pd.DataFrame:
    fund = _get_json(f"{BASE_URL}/fundamentals/{ticker}?api_token={API_TOKEN}&fmt=json")

    today_dt = _dt.datetime.now()
    hist_url = (
        f"{BASE_URL}/eod/{ticker}?api_token={API_TOKEN}"
        f"&from={(today_dt.replace(year=today_dt.year-6)).strftime('%Y-%m-%d')}"
        f"&to={today_dt.strftime('%Y-%m-%d')}&adjusted=1&fmt=json"
    )
    dfp = _pd.DataFrame(_get_json(hist_url))
    if dfp.empty:
        dfp = _pd.DataFrame(columns=["date", "adjusted_close"])
    dfp["date"] = _pd.to_datetime(dfp["date"])

    curr_price     = CurrentPrice(ticker)
    all_time_high  = ATH(ticker)
    recent_high    = dfp.loc[dfp["date"] > _pd.Timestamp(today_dt) - _pd.Timedelta(days=180),
                             "adjusted_close"].max(skipna=True)
    high_5y        = dfp["adjusted_close"].max(skipna=True)

    hi        = fund.get("Highlights", {})
    eps_ttm   = hi.get("DilutedEpsTTM")
    pe_api    = hi.get("PERatio")

    hist_raw  = fund.get("Earnings", {}).get("History", {})
    earnings  = list(hist_raw.values()) if isinstance(hist_raw, dict) else (hist_raw or [])
    eps_year  = _annual_eps_dict(fund) or _sum_quarterly_eps(earnings)
    pe_series = _five_year_pe_series(dfp, eps_year)
    avg_pe5y  = sum(pe_series) / len(pe_series) if len(pe_series) >= 2 else None

    if pe_api is None and eps_ttm and eps_ttm != 0:
        last_px = dfp.iloc[-1]["adjusted_close"] if not dfp.empty else curr_price
        pe_api  = last_px / eps_ttm

    eps_est   = hi.get("EPSEstimateNextYear")
    fair_val  = avg_pe5y * eps_ttm if avg_pe5y and eps_ttm else None
    fair_valF = avg_pe5y * eps_est if avg_pe5y and eps_est else None

    df_tech = get_historical_prices(ticker, start=(today_dt.replace(year=today_dt.year - 2)).strftime('%Y-%m-%d'))
    real_price, price_time = get_current_price(ticker)
    df_tech = append_today_price(df_tech, real_price)

    macd_df = calculate_macd(df_tech)
    rsi_df = calculate_rsi_wilder(df_tech)

    macd_raw = macd_df['MACD'].iloc[-1]
    macd_signal = macd_df['Signal'].iloc[-1]
    rsi_1y = rsi_df['RSI'].iloc[-1]

    row = {
        "Ticker":             _strip_suffix(ticker),
        "Current Price":      curr_price,
        "All-Time High":      all_time_high,
        "ATH %Chg":           _pct_from_high(curr_price, all_time_high),
        "6-Month High":       recent_high,
        "Recent-High %Chg":   _pct_from_high(curr_price, recent_high),
        "5-Year High":        high_5y,
        "Current P/E":        pe_api,
        "Avg 5-Year P/E":     avg_pe5y,
        "Fair Value (TTM)":   fair_val,
        "Fair Value %Chg":    _pct_to_value(curr_price, fair_val),
        "Avg P/E × Fwd EPS":  fair_valF,
        "MACD (12,26,9)":     macd_raw,
        "MACD Signal":        macd_signal,
        "RSI (1 yr)":         rsi_1y,
        "Live Price Time":    price_time
    }
    return _pd.DataFrame([row])

def FundamentalsBuilder(tickers):
    return _pd.concat([Fundamentals(t) for t in tickers], ignore_index=True)

def SaveToCSV(df, fname):
    df.to_csv(fname, index=False)
    print(f"Saved {len(df)} rows → {fname}")

# if __name__ == "__main__":
#     raw1 = ['msft', 'aapl', 'goog']
#     tickers = [f"{t}.US" for t in raw1]
#     df1 = FundamentalsBuilder(tickers)
#     # display(df1)
#     # SaveToCSV(df1, "fundamentals_eod7.csv")
